# Annotate w1118_42D cells with cell type labels

In [ ]:
!python --version

## Load required packages 

In [ ]:
import numpy as np
import pandas as pd
import anndata as ad
import scanpy as sc
import scvi
import torch
import scarches as sca
from scarches.dataset.trvae.data_handling import remove_sparsity
from scipy.sparse import csr_matrix
from scipy.stats import median_abs_deviation
import seaborn as sns
from pathlib import Path
import matplotlib.pyplot as plt
import gdown
import copy as cp


In [ ]:
sns.set_style('white')
sns.set(font_scale=1.5)
sc.settings.set_figure_params(dpi=80, facecolor="white")
sc.logging.print_header()
sc.settings.verbosity = 3

## Read FCA reference data and w1118_42d data

In [ ]:

## Set up I/O directories
basepath = Path("/projectnb/mccall/sbandyadka/drpr42d_snrnaseq/")
inputpath = basepath.joinpath('analysis','preprocess')
referencepath = basepath.joinpath('reference','FCA_head')
outputpath = basepath.joinpath('analysis','scarches')
outputfile = basepath.joinpath('analysis','scarches','w1118_42D_annotated.h5ad')


In [ ]:
## Read in count matrix
#w1118_42d = sc.read_10x_mtx(
#    inputpath.joinpath('w1118_42D','outs','filtered_feature_bc_matrix'),
#    var_names="gene_symbols",  
#    cache=True)

## Read sctk-qc h5ad 
w1118_42d = sc.read_h5ad(inputpath.joinpath("w1118_42d.h5ad"))
w1118_42d.raw = w1118_42d

w1118_42d

In [ ]:
## Read reference h5ad 
fca_reference = sc.read_h5ad(referencepath.joinpath("v2_fca_biohub_head_10x_raw.h5ad"))
fca_reference


In [ ]:
sc.pl.umap(fca_reference,color="annotation")

In [ ]:
fca_reference.raw = fca_reference
sc.pp.normalize_total(fca_reference)
sc.pp.log1p(fca_reference)



In [ ]:
fca_reference.obs[:5]

In [ ]:
sc.pp.highly_variable_genes(
    fca_reference,
    n_top_genes=2000,
    batch_key="batch",
    subset=True)

In [ ]:
sca.models.SCVI.setup_anndata(fca_reference, batch_key="batch")

In [ ]:
import torch

In [ ]:
vae = sca.models.SCVI(
    fca_reference,
    n_layers=2,
    encode_covariates=True,
    deeply_inject_covariates=True,
    use_layer_norm="both",
    use_batch_norm="none"
)

In [ ]:
vae.train(max_epochs=80)

In [2]:
import os
print(os.getenv("CUDA_VISIBLE_DEVICES"))

0


In [7]:
import torch

print(torch.cuda.is_available())
print(torch.cuda.device_count())
print(torch.cuda.current_device())
print(torch.cuda.device(0))
print(torch.cuda.get_device_name(0))


True
1
0
Tesla V100-PCIE-16GB


In [8]:
!nvcc --version

/bin/bash: nvcc: command not found
